In [1]:
from pyspark.sql import SparkSession
import getpass 
username = getpass.getuser()
spark = SparkSession. \
     builder. \
     config('spark.ui.port', '0'). \
     config("spark.sql.warehouse.dir", f"/user/itv009198/warehouse"). \
     config('spark.shuffle.useOldFetchProtocol', 'true'). \
     enableHiveSupport(). \
     master('yarn'). \
     getOrCreate()

In [2]:
loans_def_raw_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")

In [3]:
loans_def_raw_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
9cb79aa7323e81be1...,2.0,0.0,0.0,0.0,0.0,0.0,11.0,null
0dd2bbc517e3c8f9e...,0.0,0.0,1.0,1.0,3.0,0.0,null,115.0
458458599d3df3bfc...,0.0,0.0,1.0,1.0,1.0,0.0,null,76.0
05ea141ec28b5c7f7...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
aac68850fdac09fd0...,1.0,0.0,0.0,0.0,0.0,0.0,21.0,null
3a423e4589e89f429...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
f1efcf7dfbfef21be...,0.0,0.0,0.0,0.0,1.0,0.0,null,null
c89986155a070db2e...,1.0,0.0,0.0,0.0,1.0,15.0,5.0,null
118dc629b6e134419...,0.0,0.0,0.0,0.0,0.0,0.0,null,null
a86fa4b7493708333...,0.0,0.0,0.0,0.0,0.0,0.0,null,null


In [4]:
loans_def_raw_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- total_rec_late_fee: string (nullable = true)
 |-- mths_since_last_delinq: string (nullable = true)
 |-- mths_since_last_record: string (nullable = true)



In [5]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [6]:
spark.sql("select distinct(delinq_2yrs)from loan_defaulters")

delinq_2yrs
1.0
I bike to work on...
271 monthly payme...
183xx
VISA and AMEX cre...
etc. and I feel t...
AZ
017xx
923xx
446xx


In [7]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc ").show(40)

+------------------+-------+
|       delinq_2yrs|  total|
+------------------+-------+
|               0.0|1838878|
|               1.0| 281335|
|               2.0|  81285|
|               3.0|  29539|
|               4.0|  13179|
|               5.0|   6599|
|               6.0|   3717|
|               7.0|   2062|
|               8.0|   1223|
|               9.0|    818|
|              10.0|    556|
|              11.0|    363|
|              12.0|    264|
|              13.0|    165|
|              14.0|    120|
|              15.0|     87|
|              null|     63|
|              16.0|     55|
|              17.0|     30|
|              18.0|     30|
|              19.0|     23|
|              20.0|     17|
|              21.0|     12|
|                CA|      8|
|                TX|      6|
|                IL|      5|
|              22.0|      5|
|debt_consolidation|      5|
|    small_business|      5|
|                FL|      4|
|              24.0|      4|
|             

In [8]:
loan_defaulters_schema = "member_id string,delinq_2yrs float,delinq_amnt float,pub_rec float,pub_rec_bankruptcies float,inq_last_6mths float,total_rec_late_fee float,mths_since_last_delinq float,mths_since_last_record float"

In [9]:
loans_def_raw_df = spark.read \
.format("csv") \
.option("header", True) \
.schema(loan_defaulters_schema) \
.load("/public/trendytech/lendingclubproject/raw/loans_defaulters_csv")

In [10]:
loans_def_raw_df.createOrReplaceTempView("loan_defaulters")

In [11]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc ").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|        0.0|1838878|
|        1.0| 281335|
|        2.0|  81285|
|        3.0|  29539|
|        4.0|  13179|
|        5.0|   6599|
|        6.0|   3717|
|        7.0|   2062|
|        8.0|   1223|
|        9.0|    818|
|       10.0|    556|
|       11.0|    363|
|       12.0|    264|
|       null|    261|
|       13.0|    165|
|       14.0|    120|
|       15.0|     87|
|       16.0|     55|
|       17.0|     30|
|       18.0|     30|
|       19.0|     23|
|       20.0|     17|
|       21.0|     12|
|       22.0|      5|
|       24.0|      4|
|       26.0|      3|
|       25.0|      2|
|       29.0|      2|
|       3.44|      2|
|       23.0|      2|
|       30.0|      2|
|      10.92|      1|
|       58.0|      1|
|      18.53|      1|
|       1.41|      1|
|      26.24|      1|
|       6.52|      1|
|       5.52|      1|
|      17.18|      1|
|       9.44|      1|
+-----------+-------+
only showing top 40 rows



In [12]:
from pyspark.sql.functions import col

In [13]:
loans_def_processed_df = loans_def_raw_df.withColumn("delinq_2yrs", col("delinq_2yrs").cast("integer")).fillna(0, subset=["delinq_2yrs"])


In [14]:
loans_def_processed_df.createOrReplaceTempView("loan_defaulters")

In [15]:
spark.sql("select count(*) from loan_defaulters where delinq_2yrs is null")

count(1)
0


In [16]:
spark.sql("select delinq_2yrs, count(*) as total from loan_defaulters group by delinq_2yrs order by total desc ").show(40)

+-----------+-------+
|delinq_2yrs|  total|
+-----------+-------+
|          0|1839141|
|          1| 281337|
|          2|  81285|
|          3|  29545|
|          4|  13180|
|          5|   6601|
|          6|   3719|
|          7|   2063|
|          8|   1226|
|          9|    821|
|         10|    558|
|         11|    363|
|         12|    266|
|         13|    167|
|         14|    123|
|         15|     90|
|         16|     56|
|         17|     33|
|         18|     32|
|         19|     24|
|         20|     19|
|         21|     16|
|         22|      7|
|         24|      6|
|         23|      5|
|         26|      4|
|         25|      2|
|         30|      2|
|         29|      2|
|         27|      1|
|         35|      1|
|         28|      1|
|         42|      1|
|         39|      1|
|         32|      1|
|         36|      1|
|         58|      1|
+-----------+-------+



In [17]:
loans_def_delinq_df = spark.sql("select member_id, delinq_2yrs, delinq_amnt, int(mths_since_last_delinq) from loan_defaulters where delinq_2yrs > 0.0 or mths_since_last_delinq > 0.0")


In [18]:
loans_def_delinq_df

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
9cb79aa7323e81be1...,2,0.0,11
aac68850fdac09fd0...,1,0.0,21
c89986155a070db2e...,1,0.0,5
6e8d94bf446e97025...,0,0.0,36
42f73fd8a01f1c475...,0,0.0,46
1eef79a0e79b72c7a...,1,0.0,21
1dd1d1b51473d4993...,0,0.0,44
ec1953dba2cfb89ad...,2,0.0,13
8241a6bb3a9350fb8...,0,0.0,57
cdc94fa1c29a6a70a...,0,0.0,44


In [19]:
loans_def_record_enq_df = spark.sql("select member_id from loan_defaulters where pub_rec > 0.0 or pub_rec_bankruptcies > 0.0 or inq_last_6mths > 0.0")

In [20]:
loans_def_record_enq_df

member_id
0dd2bbc517e3c8f9e...
458458599d3df3bfc...
f1efcf7dfbfef21be...
c89986155a070db2e...
e88945f86a96f8d71...
4e1c30a5dfe9f1e20...
76cbefe31f7834f47...
47d002f59a274c6f2...
09a1c6855801dad88...
56d4375718ad6940d...


In [21]:
loans_def_delinq_df.write \
    .option("header", True) \
    .format("csv") \
    .mode("overwrite") \
    .option("path","public/trendytech/datasets/lendingclubproject/cleaned/loans_defaulters_delinq_csv") \
    .save()

In [22]:
loans_def_delinq_df

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
9cb79aa7323e81be1...,2,0.0,11
aac68850fdac09fd0...,1,0.0,21
c89986155a070db2e...,1,0.0,5
6e8d94bf446e97025...,0,0.0,36
42f73fd8a01f1c475...,0,0.0,46
1eef79a0e79b72c7a...,1,0.0,21
1dd1d1b51473d4993...,0,0.0,44
ec1953dba2cfb89ad...,2,0.0,13
8241a6bb3a9350fb8...,0,0.0,57
cdc94fa1c29a6a70a...,0,0.0,44


In [23]:
loans_def_delinq_df.write \
.option("header", True) \
.format("parquet") \
.mode("overwrite") \
.option("path","public/trendytech/datasets/lendingclubproject/cleaned/loans_defaulters_delinq_parquet") \
.save()

In [24]:
loans_def_delinq_df

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
9cb79aa7323e81be1...,2,0.0,11
aac68850fdac09fd0...,1,0.0,21
c89986155a070db2e...,1,0.0,5
6e8d94bf446e97025...,0,0.0,36
42f73fd8a01f1c475...,0,0.0,46
1eef79a0e79b72c7a...,1,0.0,21
1dd1d1b51473d4993...,0,0.0,44
ec1953dba2cfb89ad...,2,0.0,13
8241a6bb3a9350fb8...,0,0.0,57
cdc94fa1c29a6a70a...,0,0.0,44


In [25]:
loans_def_delinq_df

member_id,delinq_2yrs,delinq_amnt,mths_since_last_delinq
9cb79aa7323e81be1...,2,0.0,11
aac68850fdac09fd0...,1,0.0,21
c89986155a070db2e...,1,0.0,5
6e8d94bf446e97025...,0,0.0,36
42f73fd8a01f1c475...,0,0.0,46
1eef79a0e79b72c7a...,1,0.0,21
1dd1d1b51473d4993...,0,0.0,44
ec1953dba2cfb89ad...,2,0.0,13
8241a6bb3a9350fb8...,0,0.0,57
cdc94fa1c29a6a70a...,0,0.0,44


In [26]:
loans_def_record_enq_df.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path","public/trendytech/datasets/lendingclubproject/cleaned/loans_defaulters_records_enq_csv") \
.save()

In [27]:
loans_def_record_enq_df

member_id
0dd2bbc517e3c8f9e...
458458599d3df3bfc...
f1efcf7dfbfef21be...
c89986155a070db2e...
e88945f86a96f8d71...
4e1c30a5dfe9f1e20...
76cbefe31f7834f47...
47d002f59a274c6f2...
09a1c6855801dad88...
56d4375718ad6940d...


In [28]:
loans_def_record_enq_df.write \
.option("header", True) \
.format("parquet") \
.mode("overwrite") \
.option("path","public/trendytech/datasets/lendingclubproject/cleaned/loans_defaulters_records_enq_parquet") \
.save()

In [29]:
loans_def_record_enq_df

member_id
0dd2bbc517e3c8f9e...
458458599d3df3bfc...
f1efcf7dfbfef21be...
c89986155a070db2e...
e88945f86a96f8d71...
4e1c30a5dfe9f1e20...
76cbefe31f7834f47...
47d002f59a274c6f2...
09a1c6855801dad88...
56d4375718ad6940d...


In [30]:
loans_def_p_pub_rec_df = loans_def_processed_df.withColumn("pub_rec", col("pub_rec").cast("integer")).fillna(0, subset=["pub_rec"])

In [31]:
loans_def_p_pub_rec_df

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
9cb79aa7323e81be1...,2,0.0,0,0.0,0.0,0.0,11.0,null
0dd2bbc517e3c8f9e...,0,0.0,1,1.0,3.0,0.0,null,115.0
458458599d3df3bfc...,0,0.0,1,1.0,1.0,0.0,null,76.0
05ea141ec28b5c7f7...,0,0.0,0,0.0,0.0,0.0,null,null
aac68850fdac09fd0...,1,0.0,0,0.0,0.0,0.0,21.0,null
3a423e4589e89f429...,0,0.0,0,0.0,0.0,0.0,null,null
f1efcf7dfbfef21be...,0,0.0,0,0.0,1.0,0.0,null,null
c89986155a070db2e...,1,0.0,0,0.0,1.0,15.0,5.0,null
118dc629b6e134419...,0,0.0,0,0.0,0.0,0.0,null,null
a86fa4b7493708333...,0,0.0,0,0.0,0.0,0.0,null,null


In [32]:
loans_def_p_pub_rec_bankruptcies_df = loans_def_p_pub_rec_df.withColumn("pub_rec_bankruptcies", col("pub_rec_bankruptcies").cast("integer")).fillna(0, subset=["pub_rec_bankruptcies"])

In [33]:
loans_def_p_inq_last_6mths_df = loans_def_p_pub_rec_bankruptcies_df.withColumn("inq_last_6mths", col("inq_last_6mths").cast("integer")).fillna(0, subset=["inq_last_6mths"])

In [34]:
loans_def_p_inq_last_6mths_df 

member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,total_rec_late_fee,mths_since_last_delinq,mths_since_last_record
9cb79aa7323e81be1...,2,0.0,0,0,0,0.0,11.0,null
0dd2bbc517e3c8f9e...,0,0.0,1,1,3,0.0,null,115.0
458458599d3df3bfc...,0,0.0,1,1,1,0.0,null,76.0
05ea141ec28b5c7f7...,0,0.0,0,0,0,0.0,null,null
aac68850fdac09fd0...,1,0.0,0,0,0,0.0,21.0,null
3a423e4589e89f429...,0,0.0,0,0,0,0.0,null,null
f1efcf7dfbfef21be...,0,0.0,0,0,1,0.0,null,null
c89986155a070db2e...,1,0.0,0,0,1,15.0,5.0,null
118dc629b6e134419...,0,0.0,0,0,0,0.0,null,null
a86fa4b7493708333...,0,0.0,0,0,0,0.0,null,null


In [35]:
loans_def_p_inq_last_6mths_df.createOrReplaceTempView("loan_defaulters")

In [36]:
loans_def_detail_records_enq_df = spark.sql("select member_id, pub_rec, pub_rec_bankruptcies, inq_last_6mths from loan_defaulters")

In [37]:
loans_def_detail_records_enq_df.write \
    .option("header", True) \
    .format("csv") \
    .mode("overwrite") \
    .option("path","public/trendytech/datasets/lendingclubproject/cleaned/loans_defaulters_detail_records_enq_csv") \
    .save()

In [38]:
loans_def_detail_records_enq_df

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
9cb79aa7323e81be1...,0,0,0
0dd2bbc517e3c8f9e...,1,1,3
458458599d3df3bfc...,1,1,1
05ea141ec28b5c7f7...,0,0,0
aac68850fdac09fd0...,0,0,0
3a423e4589e89f429...,0,0,0
f1efcf7dfbfef21be...,0,0,1
c89986155a070db2e...,0,0,1
118dc629b6e134419...,0,0,0
a86fa4b7493708333...,0,0,0


In [39]:
loans_def_detail_records_enq_df.write \
    .option("header", True) \
    .format("parquet") \
    .mode("overwrite") \
    .option("path","public/trendytech/datasets/lendingclubproject/cleaned/loans_defaulters_detail_records_enq_parquet") \
    .save()

In [40]:
loans_def_detail_records_enq_df 

member_id,pub_rec,pub_rec_bankruptcies,inq_last_6mths
9cb79aa7323e81be1...,0,0,0
0dd2bbc517e3c8f9e...,1,1,3
458458599d3df3bfc...,1,1,1
05ea141ec28b5c7f7...,0,0,0
aac68850fdac09fd0...,0,0,0
3a423e4589e89f429...,0,0,0
f1efcf7dfbfef21be...,0,0,1
c89986155a070db2e...,0,0,1
118dc629b6e134419...,0,0,0
a86fa4b7493708333...,0,0,0
